In [4]:
import rasterio as rs
import rasterstats as rstats
import geopandas as gp
import pandas as pd
import re

In [2]:
countries_geojson = gp.read_file("data/country/world_countries.json")
countries_geojson

,Country,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."
...,...,...,...
250,Samoa,WSM,"MULTIPOLYGON (((-171.57002 -13.93816, -171.564..."
251,Yemen,YEM,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12..."
252,South Africa,ZAF,"MULTIPOLYGON (((37.86378 -46.94085, 37.83644 -..."
253,Zambia,ZMB,"POLYGON ((31.11984 -8.61663, 31.14102 -8.60619..."


In [5]:
def retrieveGeoStats(raster_file):
    year = re.search(r"\d+", raster_file).group()
    rf = rs.open(raster_file,"r")
    values = pd.DataFrame(rstats.zonal_stats(countries_geojson, rf.read(1), affine=rf.transform, stats=["count", "mean", "min", "max", 'median', 'std', "majority"], geojson_out=True, nodata=-1))
    del rf
    return countries_geojson.merge(pd.concat([values, values.properties.apply(pd.Series)], axis=1).drop(columns=["geometry", "properties"])).rename(columns={"mean":"average", "majority":"mode"}).set_index("Country").rename(columns=lambda name: f"{name}LightPollution{year}").rename(columns={f"countLightPollution{year}":f"recordCount{year}"}).reset_index()

nightLight1992 = retrieveGeoStats("data/nightLight/Harmonized_DN_NTL_1992_calDMSP.tif")
nightLight1992